## Power Capacity Planning

- A regional power company is devising an energy generation strategy for the next decade. They currently provide power to two adjacent counties, Avery and Burke, using a mix of nuclear, coal, and solar energy.
<br/><br/>
- Avery and Burke Counties are both growing rapidly; the company predicts that future demand for energy in the region will exceed the current grid's capacity by up to $5.0 \times 10^3$ megawatts (MW) at peak times.
<br/><br/>
- Costs of developing additional grid capacity in each county (based on existing facilities and infrastructure) are provided in the table below, all given in dollars per MW: 

| County\Type | Nuclear | Coal | Solar |
| ----------- | ------- | ---- | ----- |
| Avery | $5.2 \times 10^6$ | $2.5 \times 10^6$ | $8.0 \times 10^6$ |
| Burke | $4.8 \times 10^6$ | $2.2 \times 10^6$ | $8.5 \times 10^6$ |

- Greenhouse gas emission rates (in tons of gas per megawatt generated) for nuclear, coal, and solar power generation are $1.5\times 10^3$ tons/MW, $5.3 \times 10^3$ tons/MW, and $0.1 \times 10^3$ tons/MW, respectively. The power company has pledged to cap total additional greenhouse gas emissions resulting from this capacity expansion project at $7.2 \times 10^6$ tons.
<br/><br/>
- Burke County voters and politicians have been explicit about what they want their role in this capacity expansion to look like:
  - No additional capacity supplied by nuclear power from Burke County,
  - No more than $1.5 \times 10^3$ MW of additional capacity supplied by coal power from Burke County, and
  - No less than $2 \times 10^3$ MW of additional capacity supplied by solar power from Burke County.
<br/><br/>
- **Goal**: Find a mix of nuclear, coal, and solar energy expansion in each county that satisfies all requirements at minimum cost.


- Define Types = {Nuclear, Coal, Solar} and Counties = {Avery, Burke}.
- Define parameters:
    - $c_{ij}$: The cost per 1000 MW of power for each $i \in$ Types, $j \in$ Counties.
    - $e_{i}$: Emissions level per 1000 MW for each $i \in$ Types.
    - $e_{\max}$: Emissions ceiling in pounds per 1000 MW.
    - $d_{\max}$: Maximum amount of power demanded over current capacity, in thousands of MW.
    - $\ell_{ij},\ u_{ij}$: Lower and upper bounds for added capacity in thousands of MW for each $i \in$ Types, $j \in$ Counties (may be $\pm \infty$)
- The optimization problem is:
\begin{align*} 
\underset{\mathbf{x}}{\min} \quad  &\sum_{i \in \text{Types}} \sum_{j \in \text{Counties}} c_{ij} x_{ij} \\
\text{s.t.} \ \ \quad &\sum_{i \in \text{Types}} \sum_{j \in \text{Counties}} x_{ij}  \geq d_{\max} \\
&\sum_{i \in \text{Types}} e_i \left[\sum_{j \in \text{Counties}} x_{ij}\right] \leq e_{\max} \\
&x_{ij} \in [\ell_{ij}, u_{ij}] \quad \forall i\in\text{Types},\ j \in \text{Counties}
\end{align*}

Let's define the data in code.

In [3]:
import Pkg
Pkg.add("NamedArrays")

   Resolving package versions...
   Installed Combinatorics ─── v1.0.2
   Installed InvertedIndices ─ v1.3.1
   Installed NamedArrays ───── v0.10.3
   Installed DelimitedFiles ── v1.9.1
   Installed Requires ──────── v1.3.0
    Updating `C:\Users\dz28l\.julia\environments\v1.11\Project.toml`
  [86f7a689] + NamedArrays v0.10.3
    Updating `C:\Users\dz28l\.julia\environments\v1.11\Manifest.toml`
  [861a8166] + Combinatorics v1.0.2
  [8bb1440f] + DelimitedFiles v1.9.1
  [41ab1584] + InvertedIndices v1.3.1
  [86f7a689] + NamedArrays v0.10.3
  [ae029012] + Requires v1.3.0
Precompiling project...
    644.2 ms  ✓ InvertedIndices
    733.4 ms  ✓ Requires
    726.5 ms  ✓ DelimitedFiles
   1331.6 ms  ✓ Combinatorics
   1562.0 ms  ✓ NamedArrays
  5 dependencies successfully precompiled in 4 seconds. 78 already precompiled.


In [7]:
types = [:nuclear, :coal, :solar]
counties = [:Avery, :Burke]

dmax = 5
emax = 7.2*10^3

using NamedArrays
#Remember that the x variables will be in units of 10^3 MW 
cmat = [5.2*10^3 4.8*10^3; 2.5*10^3 2.25*10^3; 8*10^3 8.5*10^3]
c = NamedArray( cmat, (types,counties), ("type","county") )

e = Dict(zip(types,[1.5,5.3,0.1]))

umat = [Inf 0; Inf 1.5; Inf Inf]
u = NamedArray( umat, (types,counties), ("type","county") )

lmat = [0 0; 0 0; 0 2]
l = NamedArray( lmat, (types,counties), ("type","county") )

u

3×2 Named Matrix{Float64}
type ╲ county │ Avery  Burke
──────────────┼─────────────
nuclear       │   Inf    0.0
coal          │   Inf    1.5
solar         │   Inf    Inf

Now, let's define the model.

In [10]:
using JuMP
using HiGHS

#Define model object
power = Model(HiGHS.Optimizer)

#Variable definitions including both lower and upper bounds
@variable(power, l[i,j] <= x[i in types, j in counties] <= u[i,j]) 

#Additional Demand constraint
@constraint(power, demand, sum(sum(x[i,j] for j in counties) for i in types)) #This is wrong rnw lol

#Emissions Constraint
@constraint(power, emissions, sum(e[i] * sum(x[i,j] for j in counties) for i in types <=) <= emax)

#Objective Function
@objective(power, Min, sum(sum(c[i,j]*x[i,j] for j in counties) for i in types))

print(power)

LoadError: LoadError: At In[10]:11: `@constraint(power, demand, sum((sum((x[i, j] for j = counties)) for i = types)))`: Unsupported constraint expression: we don't know how to parse constraints containing the operator sum.

If you are writing a JuMP extension, implement `parse_constraint_call(::Function, ::Bool, ::Val{sum}, args...)
in expression starting at In[10]:11

In [ ]:
optimize!(power)

In [ ]:
@show objective_value(power);
@show value.(x);